In [4]:
import pandas as pd 
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [117]:
f = open('users.json')
users = json.load(f)
users = pd.DataFrame(users['users']).pop('id')
users

0      dhgfuvwifboe
1    dhfbkuhkvfhkoe
2    dfyuhiobgjvgxw
Name: id, dtype: object

In [92]:
f = open('wishlist.json')
wishlist = json.load(f)
wishlist = pd.DataFrame(wishlist['products'])
wishlist 

,id,type,brand,model
0,hifuvovuosaci,CPU,AMD,Ryzen 9 5950X Processor
1,hifuvovuosaci,CPU,AMD,Ryzen 5 5600X Processor
2,hifuvovuosaci,CPU,AMD,Ryzen 7 5800X Processor


In [8]:
#for each product in the wishlist make request to api and get the
#products full description
f = open('wishlist-products.json')
wishlist_products = json.load(f)
wishlist_description = pd.DataFrame(wishlist_products['products'])#.pop('description')
wishlist_description.drop(['model','image', 'link', 'availability','retailer' ], axis=1, inplace=True)

In [29]:
modify = re.compile(r"\([^,{}]*\)")
def clean_data(text):
    #print(list(text))
    final = [(x +"-" + y).lower() for x,y in zip(list(text), list(text.values()))]
    return "  ".join(final)
#     text = text.lower()
#     text = modify.sub("", text)
    
wishlist_description['description_cl'] = wishlist_description['description'].apply(clean_data)
wishlist_description.drop(['description'], axis=1, inplace=True)
wishlist_description
#w = list(wishlist_description['description'])


,id,brand,price,type,description_cl
0,7b05c566-be35-4829-9373-e36367e20937,AMD,11299,cpu,# of cpu cores-16 # of threads-32 base clock...
1,7b05c566-be35-4829-9373-e36367e20937,AMD,11299,cpu,# of cpu cores-6 # of threads-12 base clock-...
2,7b05c566-be35-4829-9373-e36367e20937,AMD,11299,cpu,# of cpu cores-8 # of threads-16 base clock-...


In [93]:
wishlist_description = pd.DataFrame().from_records(wishlist_description)
wishlist_description

,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,Total L2 Cache,Total L3 Cache,Unlocked,CMOS,Package,PCI ExpressÂ® Version,Thermal Solution (PIB),Default TDP / TDP
0,16,32,3.4GHz,Up to 4.9GHz,8MB,64MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Not included,105W
1,6,12,3.7GHz,Up to 4.6GHz,3MB,32MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Wraith Stealth,65W
2,8,16,3.8GHz,Up to 4.7GHz,4MB,32MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Not included,105W


In [115]:
#get all products in our database and their descriptions
f = open('products.json')
products = json.load(f)
products_description = pd.DataFrame(products['products']).pop('description')
products_description = pd.DataFrame().from_records(products_description)
tags = pd.DataFrame(products['products'])[['id','type', 'brand','retailer']]
products_description = pd.concat((tags, products_description), axis=1)
products_description.columns.values

array(['id', 'type', 'brand', 'retailer', '# of CPU Cores',
       '# of Threads', 'Base Clock', 'Max Boost Clock', 'Total L2 Cache',
       'Total L3 Cache', 'Unlocked', 'CMOS', 'Package',
       'PCI ExpressÂ® Version', 'Thermal Solution (PIB)',
       'Default TDP / TDP', 'Processor Base Frequency',
       'Max Turbo Frequency', 'Cache', 'Bus Speed', 'TDP',
       'Configurable TDP-up Frequency', 'Configurable TDP-up',
       'Configurable TDP-down Frequency', 'Configurable TDP-down',
       'IntelÂ® Thermal Velocity Boost Frequency', '# of QPI Links',
       'IntelÂ® Turbo Boost Max Technology 3.0 Frequency',
       'IntelÂ® Turbo Boost Technology 2.0 Frequency', '# of GPU Cores',
       'cTDP'], dtype=object)

In [111]:
df_products_des = products_description[['type', 'retailer', 'brand','# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
                                        'Default TDP / TDP', 'Processor Base Frequency','Max Turbo Frequency', 'TDP']]
#df_products_des.drop(df_products_des.filter(regex='Cache').columns, axis=1, inplace=True)
df_products_des

,type,retailer,brand,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,Default TDP / TDP,Processor Base Frequency,Max Turbo Frequency,TDP
0,cpu,Evetech,AMD,16,32,3.4GHz,Up to 4.9GHz,105W,NaN,NaN,NaN
1,cpu,Evetech,Intel,6,12,NaN,NaN,NaN,1.10GHz,4.70GHz,15 W
2,cpu,Evetech,Intel,4,8,NaN,NaN,NaN,4.30GHz,4.50GHz,112 W
3,cpu,Evetech,Intel,8,16,NaN,NaN,NaN,3.5GHz,5.3GHz,125 W
4,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.6GHz,65W,NaN,NaN,NaN
5,cpu,Evetech,AMD,12,24,3.8GHz,Up to 4.7GHz,105W,NaN,NaN,NaN
6,cpu,Evetech,AMD,8,16,3.9GHz,Up to 4.7GHz,105W,NaN,NaN,NaN
7,cpu,Evetech,AMD,16,32,3.5GHz,Up to 4.4GHz,180W,NaN,NaN,NaN
8,cpu,Evetech,AMD,6,12,3.7GHz,Up to 4.6GHz,65W,NaN,NaN,NaN
9,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.7GHz,105W,NaN,NaN,NaN


In [112]:
df_change = df_products_des.loc[df_products_des['Base Clock'].isnull()]
df_products_des.drop(df_change.index, inplace=True)
df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
df_change.rename({'Processor Base Frequency': 'Base Clock', 'Max Turbo Frequency': 'Max Boost Clock'}, axis=1, inplace=True)
df_change

C:\Users\lmoto\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\lmoto\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,type,retailer,brand,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,TDP
1,cpu,Evetech,Intel,6,12,1.10GHz,4.70GHz,15 W
2,cpu,Evetech,Intel,4,8,4.30GHz,4.50GHz,112 W
3,cpu,Evetech,Intel,8,16,3.5GHz,5.3GHz,125 W


In [113]:
df_products_des.drop(['Processor Base Frequency', 'Max Turbo Frequency', 'TDP'], axis=1, inplace=True)
df_products_des.rename({'Default TDP / TDP': 'TDP'}, axis=1, inplace=True)
#df_products_des = pd.merge(df_products_des, df_change, how='outer')
df_products_des

C:\Users\lmoto\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\lmoto\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,type,retailer,brand,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,TDP
0,cpu,Evetech,AMD,16,32,3.4GHz,Up to 4.9GHz,105W
4,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.6GHz,65W
5,cpu,Evetech,AMD,12,24,3.8GHz,Up to 4.7GHz,105W
6,cpu,Evetech,AMD,8,16,3.9GHz,Up to 4.7GHz,105W
7,cpu,Evetech,AMD,16,32,3.5GHz,Up to 4.4GHz,180W
8,cpu,Evetech,AMD,6,12,3.7GHz,Up to 4.6GHz,65W
9,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.7GHz,105W


In [114]:
df_products_des = pd.merge(df_products_des, df_change, how='outer')
df_products_des

,type,retailer,brand,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,TDP
0,cpu,Evetech,AMD,16,32,3.4GHz,Up to 4.9GHz,105W
1,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.6GHz,65W
2,cpu,Evetech,AMD,12,24,3.8GHz,Up to 4.7GHz,105W
3,cpu,Evetech,AMD,8,16,3.9GHz,Up to 4.7GHz,105W
4,cpu,Evetech,AMD,16,32,3.5GHz,Up to 4.4GHz,180W
5,cpu,Evetech,AMD,6,12,3.7GHz,Up to 4.6GHz,65W
6,cpu,Evetech,AMD,8,16,3.8GHz,Up to 4.7GHz,105W
7,cpu,Evetech,Intel,6,12,1.10GHz,4.70GHz,15 W
8,cpu,Evetech,Intel,4,8,4.30GHz,4.50GHz,112 W
9,cpu,Evetech,Intel,8,16,3.5GHz,5.3GHz,125 W


In [39]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['description'])

NameError: name 'ds' is not defined